In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-queries-50/test-queries-50.csv
/kaggle/input/test-queries-70-merged/test-queries-70-merged.csv
/kaggle/input/adm-b/Adim B/faiss_meta.jsonl
/kaggle/input/adm-b/Adim B/faiss.index
/kaggle/input/adm-b/Adim B/paragraphs (2).jsonl
/kaggle/input/test-queries-20/weak-supervised-labeled-20_reformatted.csv
/kaggle/input/m/ydenizdurmus/re-ranker/pytorch/default/1/re-ranker-best-sofar-%72accuracy/config.json
/kaggle/input/m/ydenizdurmus/re-ranker/pytorch/default/1/re-ranker-best-sofar-%72accuracy/tokenizer.json
/kaggle/input/m/ydenizdurmus/re-ranker/pytorch/default/1/re-ranker-best-sofar-%72accuracy/tokenizer_config.json
/kaggle/input/m/ydenizdurmus/re-ranker/pytorch/default/1/re-ranker-best-sofar-%72accuracy/model.safetensors
/kaggle/input/m/ydenizdurmus/re-ranker/pytorch/default/1/re-ranker-best-sofar-%72accuracy/special_tokens_map.json
/kaggle/input/m/ydenizdurmus/re-ranker/pytorch/default/1/re-ranker-best-sofar-%72accuracy/vocab.txt
/kaggle/input/re-ranker-last-v1/pytorch/d

In [58]:
!pip -q install faiss-cpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [59]:
import pandas as pd

df = pd.read_csv("/kaggle/input/test-queries-70-merged/test-queries-70-merged.csv")
df.head(20)


,query_id,query_text,case_id,rank_retrieval,label,evidence_preview,case_score
0,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/fsmh.m/fshhm_2092.txt,1,1,"Somut olayda, davalının eyleminin programı kul...",2.038716
1,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/fsmh.m/fshhm_2027.txt,2,2,Programın aktive edilişinde hak sahibinin rıza...,2.036981
2,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/fsmh.m/fshhm_2029.txt,3,2,Programın aktive edilişinde hak sahibinin rıza...,2.036981
3,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/fsmh.m/fshhm_2093.txt,4,1,"Somut olayda, davalının eyleminin programı kul...",2.035952
4,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/asliye.t.m/atm_12022.txt,5,2,A.Ş. arasında akıllı bisiklet kiralama sistemi...,2.027908
5,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/asliye.t.m/atm_12104.txt,6,1,D. İş sayılı dosyasından tespite geldiklerinde...,2.024033
6,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/fsmh.m/fshhm_1661.txt,7,2,Bir bilgisayar programının sahibi veya yetkili...,2.021868
7,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/fsmh.m/fshhm_1093.txt,8,1,Maddesinde belirtilen kişilerce kamuya sunuma ...,2.014043
8,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/fsmh.m/fshhm_1209.txt,9,1,Maddesinde belirtilen kişilerce kamuya sunuma ...,2.014043
9,query-1,"Bir yazılım geliştiricisi, müşteriye teslim et...",mahkeme_dataset/asliye.t.m/atm_12051.txt,10,1,"Davacı, farklı mahkemelerde çelişkili beyanlar...",1.578805


# Cell 1: Setup + File Check

In [78]:
# --- install (gerekirse) ---
# Kaggle'da çoğu zaman hazır gelir; hata alırsanız açın.
# !pip -q install faiss-cpu transformers sentencepiece torch

import os, json
import pandas as pd

# --- PATH placeholders (siz doldurun) ---
PATH_TEST_CSV      = "/kaggle/input/test-queries-70-merged/test-queries-70-merged.csv"
PATH_FAISS_INDEX   = "/kaggle/input/adm-b/Adim B/faiss.index"
PATH_FAISS_META    = "/kaggle/input/adm-b/Adim B/faiss_meta.jsonl"   # veya .json/.pkl her neyse
PATH_LEGALBERT_DIR = "/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk"
PATH_RERANKER_DIR  = "/kaggle/input/re-ranker-last-v1/pytorch/default/1/Re-ranker-last"

# --- 1) test csv okunuyor mu? ---
df = pd.read_csv(PATH_TEST_CSV)
print("test rows:", len(df))
print("cols:", list(df.columns))
print(df.head(2))

# --- 2) dosyalar var mı? ---
for p in [PATH_FAISS_INDEX, PATH_FAISS_META, PATH_LEGALBERT_DIR, PATH_RERANKER_DIR]:
    print(p, "->", "OK" if os.path.exists(p) else "MISSING")


test rows: 1400
cols: ['query_id', 'query_text', 'case_id', 'rank_retrieval', 'label', 'evidence_preview', 'case_score']
  query_id                                         query_text  \
0  query-1  Bir yazılım geliştiricisi, müşteriye teslim et...   
1  query-1  Bir yazılım geliştiricisi, müşteriye teslim et...   

                                 case_id  rank_retrieval  label  \
0  mahkeme_dataset/fsmh.m/fshhm_2092.txt               1      1   
1  mahkeme_dataset/fsmh.m/fshhm_2027.txt               2      2   

                                    evidence_preview  case_score  
0  Somut olayda, davalının eyleminin programı kul...    2.038716  
1  Programın aktive edilişinde hak sahibinin rıza...    2.036981  
/kaggle/input/adm-b/Adim B/faiss.index -> OK
/kaggle/input/adm-b/Adim B/faiss_meta.jsonl -> OK
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk -> OK
/kaggle/input/re-ranker-last-v1/pytorch/default/1/Re-ranker-last -> OK


# Cell 2: Load metadata (jsonl) + quick check


In [79]:
import json
import pandas as pd

# PATH_FAISS_META ve PATH_TEST_CSV önceki hücreden geliyor varsayıyorum

def load_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

meta_rows = load_jsonl(PATH_FAISS_META)
meta_df = pd.DataFrame(meta_rows)

print("meta rows:", len(meta_df))
print("meta cols:", list(meta_df.columns))
print(meta_df.head(2))


meta rows: 243838
meta cols: ['faiss_id', 'case_id', 'court', 'paragraph_id', 'text']
   faiss_id                                 case_id       court  paragraph_id  \
0         0  mahkeme_dataset/danistay.m/dm_3328.txt  danistay.m             0   
1         1  mahkeme_dataset/danistay.m/dm_3328.txt  danistay.m             1   

                                                text  
0  İSTEMİN KONUSU : ... Vergi Mahkemesinin ... ta...  
1  YARGILAMA SÜRECİ :\nDava konusu istem: Davacı ...  


In [80]:
import numpy as np
import faiss
import torch
from transformers import AutoTokenizer, AutoModel

# --- 1) FAISS index yükle ---
index = faiss.read_index(PATH_FAISS_INDEX)
print("faiss ntotal:", index.ntotal)

# --- 2) Embedding modeli yükle (LegalBertTurk) ---
tokenizer = AutoTokenizer.from_pretrained(PATH_LEGALBERT_DIR)
model = AutoModel.from_pretrained(PATH_LEGALBERT_DIR)
model.eval()

def mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

@torch.no_grad()
def embed_text(texts, max_len=256):
    # basit, deterministic
    enc = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )
    out = model(**enc)
    emb = mean_pool(out.last_hidden_state, enc["attention_mask"])
    emb = torch.nn.functional.normalize(emb, p=2, dim=1)  # cosine için
    return emb.cpu().numpy().astype("float32")

# --- 3) test.csv'den 1 query seç ---
df = pd.read_csv(PATH_TEST_CSV)
q = df.iloc[0]["query_text"]
q_id = df.iloc[0]["query_id"]
print("query_id:", q_id)
print("query_text:", q[:200])

# --- 4) FAISS arama ---
q_vec = embed_text([q], max_len=256)
D, I = index.search(q_vec, 5)  # top-5
top_faiss_ids = I[0].tolist()
top_scores = D[0].tolist()

print("top faiss_ids:", top_faiss_ids)
print("top scores:", top_scores)

# --- 5) meta ile eşleştirip göster ---
# meta_df önceki hücrede var varsayıyorum
hit_df = meta_df[meta_df["faiss_id"].isin(top_faiss_ids)].copy()
hit_df["retrieval_score"] = hit_df["faiss_id"].map({fid:s for fid,s in zip(top_faiss_ids, top_scores)})

# faiss sırasını korumak için
hit_df["order"] = hit_df["faiss_id"].map({fid:i for i,fid in enumerate(top_faiss_ids)})
hit_df = hit_df.sort_values("order")

print(hit_df[["faiss_id","case_id","court","paragraph_id","retrieval_score"]].head(5))
print("\n--- first snippet ---\n")
print(hit_df.iloc[0]["text"][:400])


Some weights of BertModel were not initialized from the model checkpoint at /kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


faiss ntotal: 243838
query_id: query-1
query_text: Bir yazılım geliştiricisi, müşteriye teslim ettiği mobil uygulamanın kaynak kodunun daha sonra başka bir firmaya satılıp aynı isimle piyasaya sürüldüğünü fark ediyor. Sözleşmede devre ilişkin açık hük
top faiss_ids: [112020, 70384, 63365, 68295, 76535]
top scores: [0.9387524127960205, 0.9335765242576599, 0.9335765242576599, 0.9297189116477966, 0.929280698299408]
        faiss_id                                   case_id       court  \
112020    112020  mahkeme_dataset/asliye.t.m/atm_12051.txt  asliye.t.m   
70384      70384     mahkeme_dataset/fsmh.m/fshhm_2093.txt      fsmh.m   
63365      63365     mahkeme_dataset/fsmh.m/fshhm_2092.txt      fsmh.m   
68295      68295     mahkeme_dataset/fsmh.m/fshhm_1950.txt      fsmh.m   
76535      76535     mahkeme_dataset/fsmh.m/fshhm_1014.txt      fsmh.m   

        paragraph_id  retrieval_score  
112020             5         0.938752  
70384             13         0.933577  
63365             1

In [81]:
TOPP = 200  # paragraf sayısı (istersen sonra değiştiririz)

def retrieve_paragraphs_for_query(query_text, topP=TOPP):
    q_vec = embed_text([query_text], max_len=256)
    D, I = index.search(q_vec, topP)

    faiss_ids = I[0].tolist()
    scores = D[0].tolist()

    tmp = meta_df[meta_df["faiss_id"].isin(faiss_ids)].copy()
    tmp["retrieval_score"] = tmp["faiss_id"].map({fid:s for fid,s in zip(faiss_ids, scores)})
    tmp["order"] = tmp["faiss_id"].map({fid:i for i,fid in enumerate(faiss_ids)})
    tmp = tmp.sort_values("order")

    # gerekli alanlar
    return tmp[["faiss_id","case_id","paragraph_id","text","retrieval_score"]].reset_index(drop=True)


# Re-ranker load (inference)
 We will load it this way to generate scores on Kaggle (inference only, not training)


In [82]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

rerank_tokenizer = AutoTokenizer.from_pretrained(PATH_RERANKER_DIR)
rerank_model = AutoModelForSequenceClassification.from_pretrained(PATH_RERANKER_DIR)
rerank_model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
rerank_model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# Step: Re-ranker scoring function (query + paragraph)
 This function generates scores for a query and a list of paragraphs.


In [83]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# PATH_RERANKER_DIR = ".../best_reranker_loss"  (siz doldurun)
rerank_tokenizer = AutoTokenizer.from_pretrained(PATH_RERANKER_DIR)
rerank_model = AutoModelForSequenceClassification.from_pretrained(PATH_RERANKER_DIR)
rerank_model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
rerank_model.to(device)

@torch.no_grad()
def rerank_scores(query_text, paragraph_texts, max_len=256, batch_size=16):
    scores = []
    for i in range(0, len(paragraph_texts), batch_size):
        batch_paras = paragraph_texts[i:i+batch_size]
        enc = rerank_tokenizer(
            [query_text]*len(batch_paras),
            batch_paras,
            padding=True,
            truncation=True,
            max_length=max_len,
            return_tensors="pt"
        )
        enc = {k:v.to(device) for k,v in enc.items()}
        out = rerank_model(**enc)
        # Binary / regression head fark etmez: genelde logits tek değer olur
        s = out.logits.squeeze(-1).detach().cpu().numpy().tolist()
        scores.extend(s)
    return np.array(scores, dtype=np.float32)


# Step: Generate "case score" for a single query (full pipeline)
This cell: takes 1 query → retrieves top-P paragraphs from FAISS → scores paragraphs with the re-ranker
→ aggregates scores within the same case into a single score → prints the top 10 cases.


In [84]:
import numpy as np
import pandas as pd

TOPP = 200

def case_pool_logsumexp(scores):
    # çok basit ve stabil: logsumexp
    scores = np.array(scores, dtype=np.float32)
    m = scores.max() if len(scores) else 0.0
    return float(m + np.log(np.exp(scores - m).sum())) if len(scores) else float("-inf")

def rerank_one_query(query_text, topP=TOPP):
    # 1) retrieval
    cand = retrieve_paragraphs_for_query(query_text, topP=topP)  # columns: case_id, text, retrieval_score ...

    # 2) rerank paragraph-level
    para_scores = rerank_scores(query_text, cand["text"].tolist(), max_len=256, batch_size=16)
    cand = cand.copy()
    cand["rerank_score_para"] = para_scores

    # 3) case-level pool
    case_rows = []
    for case_id, g in cand.groupby("case_id"):
        case_score = case_pool_logsumexp(g["rerank_score_para"].tolist())
        case_rows.append({
            "case_id": case_id,
            "rerank_case_score": case_score,
            "best_para_id": int(g.iloc[g["rerank_score_para"].values.argmax()]["paragraph_id"]),
        })
    case_df = pd.DataFrame(case_rows).sort_values("rerank_case_score", ascending=False)
    return case_df, cand

# TEST: test.csv'den ilk query
df_test = pd.read_csv(PATH_TEST_CSV)
q = df_test.iloc[0]["query_text"]

case_df, para_df = rerank_one_query(q, topP=200)
print(case_df.head(70))


                                      case_id  rerank_case_score  best_para_id
2    mahkeme_dataset/asliye.t.m/atm_12022.txt           3.993790             2
4    mahkeme_dataset/asliye.t.m/atm_12104.txt           3.263073            18
89      mahkeme_dataset/fsmh.m/fshhm_2093.txt           3.243819            12
119     mahkeme_dataset/fsmh.m/fshhm_5208.txt           3.230255             1
88      mahkeme_dataset/fsmh.m/fshhm_2092.txt           3.213791            10
..                                        ...                ...           ...
107     mahkeme_dataset/fsmh.m/fshhm_2859.txt           1.617337             1
8    mahkeme_dataset/asliye.t.m/atm_12147.txt           1.611215             0
64      mahkeme_dataset/fsmh.m/fshhm_1327.txt           1.543706             3
13   mahkeme_dataset/asliye.t.m/atm_12383.txt           1.543026             8
5    mahkeme_dataset/asliye.t.m/atm_12109.txt           1.534857             1

[70 rows x 3 columns]


The input to the re-ranker is tokenized as a pair (query, single paragraph text), and the total length is limited to 256 tokens, including special tokens.

This 256-token limit is not split separately between the query and the paragraph; instead, there is a single shared budget.

Therefore, the effective space available for the second text (the paragraph coming from cand["text"] via FAISS) is determined as follows:

After the query is tokenized, whatever number of tokens remains up to 256 is allocated to the paragraph. The paragraph is taken from the beginning, and any excess tokens are truncated.

Note: In Hugging Face tokenizers, for paired inputs, instead of enforcing a fixed split like “X tokens for the query and Y tokens for the paragraph,” truncation is applied according to the truncation strategy. Since your notebook uses truncation=True and max_length=256, in practice this means that the end of long paragraphs is truncated, and only the beginning of the paragraph is passed to the model.

In [87]:
import numpy as np
import pandas as pd

MAXLEN = 256
PATH_TEST_CSV = "/kaggle/input/test-queries-70-merged/test-queries-70-merged.csv"

# ---- 1) Tokenizer'ı notebook state'inden bul ----
def resolve_tokenizer():
    # notebook'ta re-ranker için genelde tokenizer değişkeni kullanılmış [file:2]
    for name in ["reranktokenizer", "tokenizer", "tok"]:
        obj = globals().get(name, None)
        if obj is not None and hasattr(obj, "__call__") and hasattr(obj, "encode"):
            return obj, name
    # bazı notebooklarda tokenizer/model dict içinde tutulabiliyor
    raise NameError(
        "Tokenizer bulunamadı. Önce re-ranker tokenizer yüklenen hücreyi çalıştır.\n"
        "Beklenen isimler: reranktokenizer / tokenizer / tok"
    )

TOK, TOK_NAME = resolve_tokenizer()
print("Using tokenizer variable:", TOK_NAME)

# ---- 2) dftest ----
dftest = pd.read_csv(PATH_TEST_CSV)

# kolon adlarını otomatik seç
if {"query_id", "query_text", "evidence_preview"}.issubset(dftest.columns):
    QIDCOL, QTEXTCOL, EVIDCOL = "query_id", "query_text", "evidence_preview"
elif {"queryid", "querytext", "evidencepreview"}.issubset(dftest.columns):
    QIDCOL, QTEXTCOL, EVIDCOL = "queryid", "querytext", "evidencepreview"
else:
    raise ValueError(f"Beklenen kolonlar yok. Mevcut kolonlar: {dftest.columns.tolist()}")

# ---- 3) Truncation analizi ----
def analyze_truncation_for_query(qtext: str, paras: list[str], maxlen: int = 256):
    enc_trunc = TOK(
        [qtext] * len(paras),
        paras,
        padding=False,
        truncation=True,
        max_length=maxlen,
        return_overflowing_tokens=True,
        return_length=True
    )

    # hızlı yol (destekleniyorsa)
    if "num_truncated_tokens" in enc_trunc:
        trunc_tokens = np.array(enc_trunc["num_truncated_tokens"], dtype=int)
        truncated_mask = trunc_tokens > 0
        kept_len = np.array(enc_trunc["length"], dtype=int)
        full_len = None
        return truncated_mask, kept_len, trunc_tokens, full_len

    # fallback: truncation=False ile gerçek length
    enc_full = TOK(
        [qtext] * len(paras),
        paras,
        padding=False,
        truncation=False,
        add_special_tokens=True
    )
    full_len = np.fromiter((len(ids) for ids in enc_full["input_ids"]), dtype=int)
    truncated_mask = full_len > maxlen
    kept_len = np.minimum(full_len, maxlen)
    trunc_tokens = (full_len - maxlen).clip(min=0)
    return truncated_mask, kept_len, trunc_tokens, full_len

rows = []
for qid, g in dftest.groupby(QIDCOL, sort=False):
    qtext = str(g[QTEXTCOL].iloc[0])
    paras = g[EVIDCOL].fillna("").astype(str).tolist()

    truncated_mask, kept_len, trunc_tokens, full_len = analyze_truncation_for_query(
        qtext=qtext,
        paras=paras,
        maxlen=MAXLEN
    )

    rows.append({
        "query_id": qid,
        "npairs": int(len(paras)),
        "nover256": int(truncated_mask.sum()),
        "pctover256": float(truncated_mask.mean()) if len(paras) else 0.0,
        "avg_kept_len": float(kept_len.mean()) if len(paras) else 0.0,
        "max_kept_len": int(kept_len.max()) if len(paras) else 0,
        "avg_trunc_tokens": float(trunc_tokens.mean()) if len(paras) else 0.0,
        "max_trunc_tokens": int(trunc_tokens.max()) if len(paras) else 0,
    })

report = pd.DataFrame(rows).sort_values(["pctover256", "nover256"], ascending=False)

overall_pct = float(report["nover256"].sum() / max(report["npairs"].sum(), 1))
print("Overall pct over 256:", overall_pct)

OUTCSV = "truncation_report_by_query.csv"
report.to_csv(OUTCSV, index=False)
print("saved:", OUTCSV, "rows:", len(report))

report.head(70)


Using tokenizer variable: tokenizer
Overall pct over 256: 0.0
saved: truncation_report_by_query.csv rows: 70


,query_id,npairs,nover256,pctover256,avg_kept_len,max_kept_len,avg_trunc_tokens,max_trunc_tokens
0,query-1,20,0,0.0,169.55,245,0.0,0
1,query-2,20,0,0.0,197.60,224,0.0,0
2,query-3,20,0,0.0,185.50,225,0.0,0
3,query-4,20,0,0.0,191.80,226,0.0,0
4,query-5,20,0,0.0,184.20,216,0.0,0
...,...,...,...,...,...,...,...,...
65,query-66,20,0,0.0,160.75,205,0.0,0
66,query-67,20,0,0.0,166.90,201,0.0,0
67,query-68,20,0,0.0,188.05,229,0.0,0
68,query-69,20,0,0.0,172.75,220,0.0,0


# Cell: Re-ranking results for all queries + CSV output


In [68]:
import numpy as np
import pandas as pd

TOPP = 1000

rows = []

query_ids = df_test["query_id"].unique().tolist()

for qid in query_ids:
    qtxt = df_test[df_test["query_id"] == qid].iloc[0]["query_text"]
    allowed = set(df_test[df_test["query_id"] == qid]["case_id"].tolist())

    # 1) retrieval topP
    cand = retrieve_paragraphs_for_query(qtxt, topP=TOPP)

    # 2) test adaylarına kırp
    cand = cand[cand["case_id"].isin(allowed)].reset_index(drop=True)

    # 3) paragraf skorla (re-ranker)
    para_scores = rerank_scores(qtxt, cand["text"].tolist(), max_len=256, batch_size=16)
    cand["rerank_score_para"] = para_scores

    # 4) case skoruna indir
    case_scores = {}
    for case_id, g in cand.groupby("case_id"):
        case_scores[case_id] = case_pool_logsumexp(g["rerank_score_para"].tolist())

    # 5) testteki 20 adayın hepsini yaz (garanti 20/20)
    # (Her case'in en az 1 paragrafı geliyor demiştin; ama yine de güvenlik için -inf koyuyoruz)
    for case_id in allowed:
        score = case_scores.get(case_id, float("-inf"))
        label = int(df_test[(df_test["query_id"] == qid) & (df_test["case_id"] == case_id)].iloc[0]["label"])
        rows.append({
            "query_id": qid,
            "case_id": case_id,
            "label": label,
            "rerank_case_score": float(score)
        })

res = pd.DataFrame(rows)

# 6) rerank sırası (1 en iyi)
res["rerank_rank"] = res.groupby("query_id")["rerank_case_score"] \
                        .rank(ascending=False, method="first") \
                        .astype(int)

res = res.sort_values(["query_id", "rerank_rank"])

out_path = "rerank_results.csv"
res.to_csv(out_path, index=False)
print("saved:", out_path, "| rows:", len(res), "| queries:", res["query_id"].nunique())
res.head(70)


saved: rerank_results.csv | rows: 1400 | queries: 70


,query_id,case_id,label,rerank_case_score,rerank_rank
13,query-1,mahkeme_dataset/asliye.t.m/atm_12022.txt,2,4.242784,1
14,query-1,mahkeme_dataset/asliye.t.m/atm_13198.txt,1,3.714765,2
1,query-1,mahkeme_dataset/fsmh.m/fshhm_2093.txt,1,3.474965,3
15,query-1,mahkeme_dataset/asliye.t.m/atm_12104.txt,1,3.469253,4
12,query-1,mahkeme_dataset/fsmh.m/fshhm_2029.txt,2,3.349660,5
...,...,...,...,...,...
237,query-12,mahkeme_dataset/danistay.m/dm_3548.txt,2,3.171051,6
220,query-12,mahkeme_dataset/danistay.m/dm_3472.txt,1,3.017527,7
234,query-12,mahkeme_dataset/yargitay.c.m/ycm_20363.txt,2,2.982013,8
221,query-12,mahkeme_dataset/yargitay.c.m/ycm_9210.txt,0,2.854176,9


# Cell: Metrics (from a single file)
 We will compute 3 metrics from rerank_results.csv: Recall@20, MRR, and nDCG@20.
 All metrics are computed per query and then averaged.


In [69]:
import numpy as np
import pandas as pd

# res = pd.read_csv("rerank_results.csv")  # eğer yukarıdan gelmiyorsa aç
# res kolonları: query_id, case_id, label, rerank_case_score, rerank_rank

K = 70

def dcg_at_k(labels, k=K):
    labels = np.asarray(labels)[:k]
    gains = (2 ** labels - 1)  # 0/1/2 için: 0,1,3
    discounts = 1.0 / np.log2(np.arange(2, len(labels) + 2))
    return float((gains * discounts).sum())

def ndcg_at_k(labels, k=K):
    dcg = dcg_at_k(labels, k)
    ideal = dcg_at_k(sorted(labels, reverse=True), k)
    return float(dcg / ideal) if ideal > 0 else 0.0

def mrr(labels, relevant_threshold=1):
    # ilk label>=threshold pozisyonu
    for i, y in enumerate(labels, start=1):
        if y >= relevant_threshold:
            return 1.0 / i
    return 0.0

def recall_at_k(labels, k=K, relevant_threshold=1):
    # query'de en az 2 emsal varsa: top-k içinde yakaladık mı?
    labels = np.asarray(labels)
    has_rel = (labels == 2).any()
    if not has_rel:
        return 0.0
    return float((labels[:k] >= relevant_threshold).any())

# query bazında sırala (rank'e göre)
res_sorted = res.sort_values(["query_id", "rerank_rank"])

per_q = []
for qid, g in res_sorted.groupby("query_id"):
    labels = g["label"].tolist()

    per_q.append({
        "query_id": qid,
        "ndcg@20": ndcg_at_k(labels, ),
        "mrr(label>=1)": mrr(labels, relevant_threshold=1),
        "recall@20(label==2)": recall_at_k(labels, 70, relevant_threshold=1),
    })

per_q = pd.DataFrame(per_q)
print("Mean nDCG@20:", per_q["ndcg@20"].mean())
print("Mean MRR:", per_q["mrr(label>=1)"].mean())
print("Mean Recall@20:", per_q["recall@20(label==2)"].mean())

per_q.head(70)


Mean nDCG@20: 0.7286101502446488
Mean MRR: 0.7908049886621316
Mean Recall@20: 0.4857142857142857


,query_id,ndcg@20,mrr(label>=1),recall@20(label==2)
0,query-1,0.882789,1.000000,1.0
1,query-10,0.781599,1.000000,1.0
2,query-11,0.949946,1.000000,0.0
3,query-12,0.789945,1.000000,1.0
4,query-13,0.830914,1.000000,0.0
...,...,...,...,...
65,query-69,0.698609,1.000000,1.0
66,query-7,0.809173,1.000000,0.0
67,query-70,0.828788,1.000000,1.0
68,query-8,0.430489,0.111111,0.0


# Step: Compare with the baseline (retrieval)
For this, we will treat the rank_retrieval(basically case scores order) order in the test CSV as the "baseline ranking" and compute the same metrics.


In [70]:
import numpy as np
import pandas as pd

df_test = pd.read_csv(PATH_TEST_CSV)
base_sorted = df_test.sort_values(["query_id", "rank_retrieval"])

K = 70

def dcg_at_k(labels, k=K):
    labels = np.asarray(labels)[:k]
    gains = (2 ** labels - 1)
    discounts = 1.0 / np.log2(np.arange(2, len(labels) + 2))
    return float((gains * discounts).sum())

def ndcg_at_k(labels, k=K):
    dcg = dcg_at_k(labels, k)
    ideal = dcg_at_k(sorted(labels, reverse=True), k)
    return float(dcg / ideal) if ideal > 0 else 0.0

def mrr(labels, relevant_threshold=1):
    for i, y in enumerate(labels, start=1):
        if y >= relevant_threshold:
            return 1.0 / i
    return 0.0

def recall_at_k(labels, k=K, relevant_threshold=1):
    labels = np.asarray(labels)
    has_rel = (labels >= relevant_threshold).any()
    if not has_rel:
        return 0.0
    return float((labels[:k] >= relevant_threshold).any())

per_q_base = []
for qid, g in base_sorted.groupby("query_id"):
    labels = g["label"].tolist()
    per_q_base.append({
        "query_id": qid,
        "ndcg@20": ndcg_at_k(labels, 70),
        "mrr(label>=1)": mrr(labels, 1),
        "recall@20(label>=1)": recall_at_k(labels, 20, 1),
    })

per_q_base = pd.DataFrame(per_q_base)
print("BASE Mean nDCG@20:", per_q_base["ndcg@20"].mean())
print("BASE Mean MRR:", per_q_base["mrr(label>=1)"].mean())
print("BASE Mean Recall@20:", per_q_base["recall@20(label>=1)"].mean())


BASE Mean nDCG@20: 0.692247133220431
BASE Mean MRR: 0.7428741496598639
BASE Mean Recall@20: 0.9857142857142858


In [71]:
import numpy as np
import pandas as pd

K = 70

def dcg_at_k(labels, k=K):
    labels = np.asarray(labels)[:k]
    gains = (2 ** labels - 1)
    discounts = 1.0 / np.log2(np.arange(2, len(labels) + 2))
    return float((gains * discounts).sum())

def ndcg_at_k(labels, k=K):
    dcg = dcg_at_k(labels, k)
    ideal = dcg_at_k(sorted(labels, reverse=True), k)
    return float(dcg / ideal) if ideal > 0 else 0.0

def mrr(labels, relevant_threshold=1):
    for i, y in enumerate(labels, start=1):
        if y >= relevant_threshold:
            return 1.0 / i
    return 0.0

def per_query_metrics(df_sorted, label_col="label"):
    rows = []
    for qid, g in df_sorted.groupby("query_id"):
        labels = g[label_col].tolist()
        rows.append({
            "query_id": qid,
            "ndcg@20": ndcg_at_k(labels, 70),
            "mrr": mrr(labels, 1),
        })
    return pd.DataFrame(rows)

# baseline (rank_retrieval)
base_sorted = df_test.sort_values(["query_id", "rank_retrieval"])
m_base = per_query_metrics(base_sorted).rename(columns={"ndcg@20":"base_ndcg@20", "mrr":"base_mrr"})

# rerank (rerank_rank)
rer_sorted = res.sort_values(["query_id", "rerank_rank"])
m_rer  = per_query_metrics(rer_sorted).rename(columns={"ndcg@20":"rer_ndcg@20", "mrr":"rer_mrr"})

m = m_base.merge(m_rer, on="query_id", how="inner")
m["delta_ndcg@20"] = m["rer_ndcg@20"] - m["base_ndcg@20"]
m["delta_mrr"] = m["rer_mrr"] - m["base_mrr"]

print("Top +10 nDCG improvements")
display(m.sort_values("delta_ndcg@20", ascending=False).head(10))

print("Top -10 nDCG drops")
display(m.sort_values("delta_ndcg@20", ascending=True).head(10))


Top +10 nDCG improvements


,query_id,base_ndcg@20,base_mrr,rer_ndcg@20,rer_mrr,delta_ndcg@20,delta_mrr
41,query-47,0.443480,0.142857,0.713012,1.000000,0.269532,0.857143
40,query-46,0.609747,0.500000,0.863897,1.000000,0.254150,0.500000
66,query-7,0.577760,0.333333,0.809173,1.000000,0.231413,0.666667
7,query-16,0.512062,0.200000,0.729623,1.000000,0.217561,0.800000
28,query-35,0.702537,0.250000,0.887551,1.000000,0.185015,0.750000
39,query-45,0.315465,0.125000,0.500000,0.333333,0.184535,0.208333
32,query-39,0.553838,0.500000,0.730968,1.000000,0.177130,0.500000
27,query-34,0.585518,1.000000,0.746706,1.000000,0.161188,0.000000
13,query-21,0.440065,0.200000,0.586869,0.500000,0.146804,0.300000
64,query-68,0.464333,0.250000,0.605943,0.500000,0.141611,0.250000


Top -10 nDCG drops


,query_id,base_ndcg@20,base_mrr,rer_ndcg@20,rer_mrr,delta_ndcg@20,delta_mrr
4,query-13,0.996801,1.000000,0.830914,1.000000,-0.165887,0.000000
21,query-29,0.430677,0.250000,0.270238,0.083333,-0.160438,-0.166667
10,query-19,0.875335,1.000000,0.728297,0.500000,-0.147038,-0.500000
65,query-69,0.820199,1.000000,0.698609,1.000000,-0.121590,0.000000
35,query-41,0.569781,0.333333,0.448645,0.142857,-0.121136,-0.190476
69,query-9,0.714280,1.000000,0.606496,0.500000,-0.107784,-0.500000
37,query-43,0.983444,1.000000,0.883210,0.500000,-0.100234,-0.500000
47,query-52,0.711829,1.000000,0.619243,1.000000,-0.092586,0.000000
18,query-26,0.875636,1.000000,0.785626,1.000000,-0.090010,0.000000
30,query-37,0.944845,1.000000,0.860741,1.000000,-0.084104,0.000000


***Aşağıdaki kısa kod, belirtilen queryler için baseline top-K ve re-ranker top-K’i (case_id + label) yan yana yazdırır. Çıktıyı buraya yapıştır, birlikte yorumlayalım.***

In [72]:
import pandas as pd

# df_test: test_candidates_locked.csv
# res: rerank_results.csv (ya da senin res dataframe'in)

FOCUS = ["query-19", "query-2", "query-16", "query-9", "query-14", "query-3"]

def show_top20(qid):
    base = (df_test[df_test["query_id"] == qid]
            .sort_values("rank_retrieval")
            [["case_id","label","rank_retrieval"]]
            .head(70)
            .rename(columns={"rank_retrieval":"rank"}))
    base["which"] = "baseline"

    rer = (res[res["query_id"] == qid]
           .sort_values("rerank_rank")
           [["case_id","label","rerank_rank"]]
           .head(70)
           .rename(columns={"rerank_rank":"rank"}))
    rer["which"] = "rerank"

    out = pd.concat([base, rer], ignore_index=True)
    print("\n====", qid, "====")
    print(out[["which","rank","label","case_id"]].to_string(index=False))

for qid in FOCUS:
    show_top20(qid)



==== query-19 ====
   which  rank  label                                    case_id
baseline     1      1  mahkeme_dataset/yargitay.c.m/ycm_4461.txt
baseline     2      1  mahkeme_dataset/yargitay.c.m/ycm_4443.txt
baseline     3      0  mahkeme_dataset/yargitay.c.m/ycm_9210.txt
baseline     4      0  mahkeme_dataset/yargitay.c.m/ycm_5070.txt
baseline     5      1    mahkeme_dataset/asliye.t.m/atm_5672.txt
baseline     6      1 mahkeme_dataset/yargitay.c.m/ycm_16537.txt
baseline     7      0 mahkeme_dataset/yargitay.c.m/ycm_18356.txt
baseline     8      0 mahkeme_dataset/yargitay.c.m/ycm_20351.txt
baseline     9      0 mahkeme_dataset/yargitay.c.m/ycm_18330.txt
baseline    10      1     mahkeme_dataset/danistay.m/dm_0533.txt
baseline    11      1 mahkeme_dataset/yargitay.c.m/ycm_14697.txt
baseline    12      0     mahkeme_dataset/danistay.m/dm_3431.txt
baseline    13      1 mahkeme_dataset/yargitay.c.m/ycm_20349.txt
baseline    14      1 mahkeme_dataset/yargitay.c.m/ycm_13280.txt
basel